In [19]:
import torch
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-uncased')

reverse_voc = {v: k for k, v in tokenizer.vocab.items()}

tokenizer

tokenizer('hello world  14  ')

{'input_ids': [101, 7592, 2088, 2403, 102], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [73]:
import torch
import torch.nn as nn
from transformers import BertModel
from tqdm import tqdm

class BertTokenClassifier(nn.Module):
    def __init__(self, bert_model_name='bert-base-uncased', hidden_dim=None, dropout=0.3):
        super(BertTokenClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        for param in self.bert.parameters():
            param.requires_grad = False
        
        bert_hidden_size = self.bert.config.hidden_size  # 768 для bert-base
        
        # Если hidden_dim не указан, используем половину размера BERT
        if hidden_dim is None:
            hidden_dim = bert_hidden_size // 2  # 384
        
        # Первый полносвязный слой: BERT hidden_size -> hidden_dim
        self.fc1 = nn.Linear(bert_hidden_size, hidden_dim)
        
        # Второй полносвязный слой: hidden_dim -> 1
        self.fc2 = nn.Linear(hidden_dim, 1)
        
        # Dropout для регуляризации
        self.dropout = nn.Dropout(dropout)
        
        # Функция активации
        self.relu = nn.ReLU()
        
    def forward(self, input_ids, attention_mask=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        token_embeddings = outputs.last_hidden_state  # (batch_size, seq_len, 768)
        
        # Первый слой с активацией
        x = self.fc1(token_embeddings)  # (batch_size, seq_len, hidden_dim)
        x = self.relu(x)  # ReLU активация
        x = self.dropout(x)  # Dropout для регуляризации
        
        # Второй слой (финальный классификатор)
        logits = self.fc2(x)  # (batch_size, seq_len, 1)
        
        return logits.squeeze(-1)  # (batch_size, seq_len)

model = BertTokenClassifier(hidden_dim=384, dropout=0.3)

# ВАЖНО: теперь нужно обучать параметры всех слоев классификатора
optimizer = torch.optim.Adam([
    {'params': model.fc1.parameters()},
    {'params': model.fc2.parameters()}
], lr=2e-5)

criterion = nn.BCEWithLogitsLoss()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

reverse_voc = {v: k for k, v in tokenizer.vocab.items()}
cls_id = tokenizer.cls_token_id  # 101
sep_id = tokenizer.sep_token_id  # 102

num_epochs = 3
batch_size = 64

for epoch in range(num_epochs):
    model.train()
    
    all_token_ids = list(tokenizer.vocab.values())
    
    for i in tqdm(range(0, len(all_token_ids), batch_size)):
        batch_ids = all_token_ids[i:i+batch_size]
        
        # Каждый токен оборачиваем в [CLS] token [SEP]
        input_ids_list = [[cls_id, tid, sep_id] for tid in batch_ids]
        attention_mask_list = [[1, 1, 1] for _ in batch_ids]
        
        # Паддинг до одинаковой длины (уже одинаковые, но для безопасности)
        max_len = 3
        input_ids_padded = []
        attention_mask_padded = []
        for seq, mask in zip(input_ids_list, attention_mask_list):
            input_ids_padded.append(seq + [0] * (max_len - len(seq)))
            attention_mask_padded.append(mask + [0] * (max_len - len(mask)))
        
        input_ids = torch.tensor(input_ids_padded).to(device)  # (batch_size, 3)
        attention_mask = torch.tensor(attention_mask_padded).to(device)  # (batch_size, 3)
        
        # Labels для каждого токена в последовательности (нам нужен только токен на позиции 1)
        labels = torch.tensor([1.0 if 'tech' in reverse_voc[tid] else 0.0 for tid in batch_ids]).to(device)  # (batch_size,)
        
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask)  # (batch_size, 3)
        
        # Берем предсказания только для токена на позиции 1 (наш целевой токен)
        token_predictions = outputs[:, 1]  # (batch_size,)
        
        loss = criterion(token_predictions, labels)
        loss.backward()
        optimizer.step()

100%|██████████| 477/477 [01:04<00:00,  7.41it/s]


In [ ]:
model.eval()
with torch.no_grad():
    x = []
    for i in range(30000):
        if 'panic' in reverse_voc[i]:
            print(reverse_voc[i])
            x.append(i)
    
    test_token_ids = x
    
    # Формируем входные данные правильно
    input_ids_list = [[cls_id, tid, sep_id] for tid in test_token_ids]
    attention_mask_list = [[1, 1, 1] for _ in test_token_ids]
    
    test_input_ids = torch.tensor(input_ids_list).to(device)  # (num_tokens, 3)
    test_attention_mask = torch.tensor(attention_mask_list).to(device)  # (num_tokens, 3)
    
    prediction_logits = model(test_input_ids, attention_mask=test_attention_mask)  # (num_tokens, 3)
    
    # Берем предсказания для токена на позиции 1
    token_logits = prediction_logits[:, 1]  # (num_tokens,)
    prediction_probs = torch.sigmoid(token_logits)
    
    print("Token IDs:", test_token_ids)
    print("Probabilities:", prediction_probs.cpu().numpy())
    predicted_labels = (prediction_probs > 0.5).long()
    print("Predicted toxic (1=toxic):", predicted_labels.cpu().numpy())

panic
hispanic
panicked
Token IDs: [6634, 6696, 16035]
Probabilities: [0.00035994 0.00032089 0.00165672]
Predicted toxic (1=toxic): [0 0 0]


In [45]:
list(reverse_voc.keys())[:1][0]

2273